# Vectorised Approach 
###             This optimeised version has a better time complexity

### Setup 

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']
tickers = tickers.to_list()

In [ ]:
df = yf.download(tickers,start= '2010-01-01', end ='2023-04-06')


### Run the above every time.^^^^^ The below is proof of concelpt

PIvot Table in Pandas (How to slice)

In [ ]:
df

In [ ]:
# This cell is the proof of concept of how slicing works before putting it in a function
#df fiiltered by( the columns filterd by (were does the lvl 1 column header value say APPLE))
df[df.columns[df.columns.get_level_values(1) == 'AAPL']]

In [ ]:
def slice_by_ticker(ticker,df):
    unpivot = df.copy()
    unpivot = unpivot[unpivot.columns[unpivot.columns.get_level_values(1) == ticker]]
    unpivot.columns = unpivot.columns.droplevel(1)
    unpivot['Price'] = unpivot['Open'].shift(-1)
    return unpivot

In [ ]:
def MA_calc(df,n,m):
    df[f'MA{n}'] = df['Adj Close'].rolling(n).mean()
    df[f'MA{m}'] = df['Adj Close'].rolling(m).mean()
    df.dropna(inplace= True)
    return df

### Building a Proof of Cocept of the Vectorised Approach using AAPL as an example 

In [ ]:
AAPL = slice_by_ticker('AAPL',df)
AAPL

In [ ]:
AAPL = MA_calc(AAPL,50,200)
AAPL

Now the DF has been set up and functions built. The vectorised approach begins

In [ ]:
# Identify the first by signal 
(AAPL.iloc[:,-2] > AAPL.iloc[:,-1]).idxmax()
# idxmax i.e. the index of the first instance where the short moving average was above the long

In [ ]:
#the first signal object is a series with the same index as the origial AAPL DF, where the value is only truen at the first by signal 
first_signal = pd.Series(AAPL.index == (AAPL.iloc[:,-2] > AAPL.iloc[:,-1]).idxmax(), index= AAPL.index)

Now to find the rest of the trade signals

In [ ]:
# Return true every time the short MA crosses the long MA
(AAPL.iloc[:,-2] > AAPL.iloc[:,-1]).diff()

In [ ]:
# Trade signals is the first signal + the above 
Trade_signals = first_signal | (AAPL.iloc[:,-2] > AAPL.iloc[:,-1]).diff()
Trade_signals

In [ ]:
Trades = AAPL[Trade_signals]
Trades

In [ ]:
len(Trades)

devise a logic that will close the position if you are currently in a trade 

In [ ]:
if len(Trades) % 2 != 0:
    temp = AAPL.tail(1).copy()
    temp['Price'] = temp['Close']
    Trades = pd.concat([Trades, temp])
len(Trades)    

In [ ]:
Trades

Now to Caculate the returns of the strategy 

In [ ]:
# this is the difference is price every i.e. flat profit 
Trades['Price'].diff()[1::2]
# run the diff function every 2nd row, starting at index 1 i.e. the 2 df entry whch is the first sale (first sale price - first by price, then skip over the next entry which is a buy to arrive at the 2nd sale. Then repeat)

Now getting returns by setting it in relation to the buy price for the trade

In [ ]:
Rets = Trades['Price'].diff()[1::2] / Trades['Price'][0::2].values
Rets

Now Cumulate returns

In [ ]:
cumret = (Rets + 1).prod()
cumret


## Now Abstracting this process into a fuctionction that can be called on all the assets in the SNP 500

In [ ]:
def slice_by_ticker(ticker,df):
    unpivot = df.copy()
    unpivot = unpivot[unpivot.columns[unpivot.columns.get_level_values(1) == ticker]]
    unpivot.columns = unpivot.columns.droplevel(1)
    unpivot['Price'] = unpivot['Open'].shift(-1)
    return unpivot

In [ ]:
def MA_calc(df,n,m):
    df[f'MA{n}'] = df['Adj Close'].rolling(n).mean()
    df[f'MA{m}'] = df['Adj Close'].rolling(m).mean()
    df.dropna(inplace= True)
    return df

In [ ]:
cumrets = []

def vectorised (ticker,df,n,m):
    #create sub data frames based on the ticker syboml and feed that 
    #to moving average function that adds the 2 moving averages to dhte df
    z = pd.DataFrame(MA_calc(slice_by_ticker(ticker, df),n,m))

    #find the first signal that's a buy 
    first_signal = pd.Series(z.index == (z.iloc[:,-2] > z.iloc[:,-1]).idxmax(), index= z.index)

    #find the rest of the signals 
    Trade_signals = first_signal | (z.iloc[:,-2] > z.iloc[:,-1]).diff()
    Trades = z[Trade_signals]
    if len(Trades) % 2 != 0:
        temp = z.tail(1).copy()
        temp['Price'] = temp['Close']
        Trades = pd.concat([Trades, temp])

    #calculate returns and cumulated return
    Rets = Trades['Price'].diff()[1::2] / Trades['Price'][0::2].values
    cumret = (Rets + 1).prod()
    return cumret


In [ ]:
cumrets = []
clean_tickers = []

for ticker in tickers:
    try:
        print(f"Cumuret for {ticker}: {vectorised(ticker, df, 10, 50)}")
        cumrets.append(vectorised(ticker, df, 10, 50))
        clean_tickers.append(ticker)
    except ValueError:
        print(f'{ticker} one Goofy ahh stock')



In [ ]:
compare = pd.DataFrame({'Results': cumrets },index= clean_tickers)
compare

In [35]:
compare['Results'].nlargest(5)

TSLA    39.031641
ALGN    29.121435
URI     27.635018
ENPH    27.488482
NVDA    26.417403
Name: Results, dtype: float64